In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torch.optim as optim
torch.cuda.empty_cache()

from torchvision.models import resnet18, ResNet18_Weights

In [2]:
from models.keypoints.KeypointPredictor import KeypointPredictor
from models.keypoints.ImageEncoder import ImageEncoder
from models.keypoints.ImageDecoder import ImageDecoder

x = torch.rand(1,3,128,128)
model = KeypointPredictor()
out, soft, m = model(x)
print(out.shape)

torch.Size([1, 100, 2])


/home/sgarikipati7/.env/ssta_env/lib/python3.12/site-packages/torchvision/models/_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(


In [3]:
x = torch.rand(1,64,32,32)
y = torch.rand(1,100,2)

model = ImageDecoder(64, 100, 3)
out = model(x,y)
out.shape

model = ImageEncoder()
out = model(torch.rand(1,3,128,128))
out.shape

torch.Size([1, 64, 32, 32])

In [4]:
import lightning as L

import matplotlib.pyplot as plt
import numpy as np
import torchvision.transforms.functional as TF
from torchvision.utils import make_grid
import io
from PIL import Image

from lightning.pytorch.callbacks import Callback

class AlphaScheduler(Callback):
    def __init__(self, warmup_epochs=10, ramp_epochs=20, final_alpha=0.01):
        """
        Args:
            warmup_epochs: Number of epochs to keep alpha = 0
            ramp_epochs: Number of epochs to linearly ramp up alpha after warmup
            final_alpha: Target alpha value
        """
        self.warmup_epochs = warmup_epochs
        self.ramp_epochs = ramp_epochs
        self.final_alpha = final_alpha

    def on_train_epoch_start(self, trainer, pl_module):
        epoch = trainer.current_epoch

        if epoch < self.warmup_epochs:
            alpha = 0.0
        elif epoch < self.warmup_epochs + self.ramp_epochs:
            # Linear ramp from 0 to final_alpha
            ramp_progress = (epoch - self.warmup_epochs) / self.ramp_epochs
            alpha = ramp_progress * self.final_alpha
        else:
            alpha = self.final_alpha

        pl_module.alpha = alpha

def draw_keypoints_on_image(image_tensor, keypoints, color='r'):
    """
    image_tensor: [3, H, W] (float in [0,1])
    keypoints: [N, 2] with (x, y) format in image coordinates
    """
    image = TF.to_pil_image(image_tensor.cpu())
    plt.figure(figsize=(4, 4))
    plt.imshow(image)
    plt.axis("off")
    for x, y in keypoints.cpu():
        plt.scatter(x, y, c=color, s=10)
    
    buf = io.BytesIO()
    plt.savefig(buf, format='jpeg', bbox_inches='tight', pad_inches=0)
    plt.close()
    buf.seek(0)
    pil_image = Image.open(buf)
    return TF.to_tensor(pil_image)
    
class LitKeypointDetector(L.LightningModule):
    def __init__(self, keypoint_encoder, feature_encoder, feature_decoder):
        super().__init__()
        self.keypoint_generator = keypoint_encoder
        self.feature_encoder = feature_encoder
        self.image_reconstructor = feature_decoder
        self.alpha = 0

    def training_step(self, batch, batch_idx):
        v1, vt = batch
        
        v1_features = self.feature_encoder(v1)

        _, _, v1_heatmaps = self.keypoint_generator(v1)
        _, vt_soft_keypoints, vt_heatmaps = self.keypoint_generator(vt)

        vt_pred = self.image_reconstructor(v1_features, vt_soft_keypoints)
        loss, mse, condens = self.keypoint_loss(vt, vt_pred, v1_heatmaps, vt_heatmaps)
        self.log("train/total_loss", loss)
        self.log("train/mse", mse)
        self.log("train/condensation_loss", condens)
        self.log("alpha", self.alpha)

        return loss
    
    def validation_step(self, batch, batch_idx):
        v1, vt = batch

        # Encode features from v0
        v1_features = self.feature_encoder(v1)

        # Get keypoints from v0 and v1 separately
        _, soft_kp_vt, v1_heatmaps = self.keypoint_generator(v1)
        _, soft_kp_vt, vt_heatmaps = self.keypoint_generator(vt)

        # Predict v1 using v0 features + v1 keypoints
        vt_pred = self.image_reconstructor(v1_features, soft_kp_vt)
        loss, mse, condens = self.keypoint_loss(vt, vt_pred, v1_heatmaps, vt_heatmaps)

        self.log("val/total_loss", loss)
        self.log("val/mse", mse)
        self.log("val/condensation_loss", condens)

        if batch_idx == 0:
            B, C, img_H, img_W = v1.shape
            _, _, heatmap_H, heatmap_W = vt_heatmaps.shape

            scale_x = img_W / heatmap_W
            scale_y = img_H / heatmap_H

            # Rescale keypoints
            scaled_kps_v0 = soft_kp_vt[0].clone()
            scaled_kps_v0[..., 0] *= scale_x
            scaled_kps_v0[..., 1] *= scale_y

            scaled_kps_v1 = soft_kp_vt[0].clone()
            scaled_kps_v1[..., 0] *= scale_x
            scaled_kps_v1[..., 1] *= scale_y

            # Clamp images for visualization
            v0_img = v1[0].clamp(0, 1)
            v1_img = vt[0].clamp(0, 1)
            v1_pred_img = vt_pred[0].clamp(0, 1)

            # Overlay keypoints
            vis_v0 = draw_keypoints_on_image(v0_img, scaled_kps_v0, color='b')  # Blue keypoints
            vis_v1 = draw_keypoints_on_image(v1_img, scaled_kps_v1, color='r')  # Red keypoints
            vis_pred = draw_keypoints_on_image(v1_pred_img, scaled_kps_v1, color='r')

            # Combine visuals
            grid = make_grid([vis_v0, vis_v1, vis_pred])
            self.logger.experiment.add_image("val/visualize_key_reconstruction", grid, self.current_epoch)

            # Log heatmaps of v1
            sample_heatmaps = vt_heatmaps[0].unsqueeze(1)  # [K, 1, H, W]
            heatmap_grid = make_grid(sample_heatmaps, nrow=10, normalize=True, scale_each=True)
            self.logger.experiment.add_image("val/heatmaps_v1", heatmap_grid, self.current_epoch)



    def configure_optimizers(self):
        optimizer = optim.AdamW(self.parameters(), lr=1e-3)
        return optimizer
    
    def keypoint_loss(self, vt, vt_pred, v0_heat, vt_heat):

        mse = nn.functional.mse_loss(vt, vt_pred)


        # condensation_loss_1 = (v0_heat.amax(dim=(-1,-2)) - v0_heat.mean(dim=(-1,-2))).mean()
        # condensation_loss_t = (vt_heat.amax(dim=(-1,-2)) - vt_heat.mean(dim=(-1,-2))).mean()
        # condensation_loss = -(condensation_loss_1 + condensation_loss_t)/2
        condensation_loss_1 = self.condensation_loss_entropy(v0_heat)
        condensation_loss_t = self.condensation_loss_entropy(vt_heat)
        condensation_loss = (condensation_loss_1 + condensation_loss_t)/2

        return mse+self.alpha*condensation_loss, mse, condensation_loss
    
    def condensation_loss_entropy(self, heatmaps):
        # heatmaps: [B, K, H, W] (softmaxed)
        eps = 1e-8
        log_h = torch.log(heatmaps + eps)
        entropy = -torch.sum(heatmaps * log_h, dim=(-2, -1))  # [B, K]
        return entropy.mean()


In [5]:
x = torch.rand(4,64,32,32)
(x.amax(dim=(-1,-2)) - x.mean(dim=(-1,-2))).mean()

tensor(0.4982)

In [6]:
from torch.utils.data import Dataset
from torchvision.datasets import MNIST
import random
import torch
from torchvision import transforms

class MovingMNISTPairs(Dataset):
    def __init__(self, root, train=True, transform=None):
        self.dataset = MNIST(root=root, train=train, download=True)
        self.transform = transform or transforms.ToTensor()
        self.canvas_size = 128
        self.digit_size = 28
        self.max_pos = self.canvas_size - self.digit_size

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        img, _ = self.dataset[idx]
        img = self.transform(img)  # [1, 28, 28]

        # Random top-left position where digit fits
        x = random.randint(0, self.max_pos)
        y = random.randint(0, self.max_pos)

        # Create black canvas and place digit
        canvas = torch.zeros(1, self.canvas_size, self.canvas_size)
        canvas[:, y:y+self.digit_size, x:x+self.digit_size] = img

        # Simulate motion (safe shift)
        dx = random.randint(-4, 4)
        dy = random.randint(-4, 4)
        new_x = min(max(x + dx, 0), self.max_pos)
        new_y = min(max(y + dy, 0), self.max_pos)

        moved = torch.zeros_like(canvas)
        moved[:, new_y:new_y+self.digit_size, new_x:new_x+self.digit_size] = img

        return canvas.expand(3, -1, -1), moved.expand(3, -1, -1)  # Convert to RGB

In [7]:
from torch.utils.data import DataLoader, random_split

def get_dataloaders(data_root="./data", batch_size=8):
    full_train = MovingMNISTPairs(data_root, train=True)
    val_size = int(0.1 * len(full_train))
    train_size = len(full_train) - val_size

    train_dataset, val_dataset = random_split(full_train, [train_size, val_size])

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=8)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=8)

    return train_loader, val_loader

In [8]:
import os
import random
from PIL import Image
from torch.utils.data import Dataset
from torchvision import transforms

class CameraSequentialPairs(Dataset):
    def __init__(self, root, transform=None, min_offset=1, max_offset=5):
        """
        Args:
            root (str): Root directory containing camera folders.
            transform: Transformations to apply to each image.
            min_offset (int): Minimum frame difference between pairs.
            max_offset (int): Maximum frame difference between pairs.
        """
        self.transform = transform or transforms.ToTensor()
        self.min_offset = min_offset
        self.max_offset = max_offset
        self.sequences = []

        # List all camera directories
        camera_dirs = sorted([os.path.join(root, d) for d in os.listdir(root)
                              if os.path.isdir(os.path.join(root, d))])
        
        # For each camera directory, get sorted image paths (using numeric sorting)
        for cam_dir in camera_dirs:
            image_dir = os.path.join(cam_dir, "images")
            if not os.path.exists(image_dir):
                continue
            image_files = sorted(
                [os.path.join(image_dir, f) for f in os.listdir(image_dir) if f.endswith('.png')],
                key=lambda x: int(os.path.splitext(os.path.basename(x))[0].split('_')[-1])
            )
            if len(image_files) > 1:
                self.sequences.append(image_files)
        
        # Build cumulative lengths for indexing purposes
        self.cumulative_lengths = []
        total = 0
        for seq in self.sequences:
            # Each valid pair comes from a starting image (all except the last)
            total += len(seq) - 1
            self.cumulative_lengths.append(total)

    def __len__(self):
        return self.cumulative_lengths[-1] if self.cumulative_lengths else 0

    def __getitem__(self, idx):
    # Find which sequence to use
        seq_idx = 0
        while idx >= self.cumulative_lengths[seq_idx]:
            seq_idx += 1
        seq = self.sequences[seq_idx]
        
        local_idx = idx if seq_idx == 0 else idx - self.cumulative_lengths[seq_idx - 1]
        
        # Check that the index is valid
        if local_idx >= len(seq) - 1:
            raise IndexError("Local index out of range for the sequence")
        
        img1_path = seq[local_idx]

        # Compute the maximum valid offset
        remaining = len(seq) - 1 - local_idx
        effective_max_offset = min(self.max_offset, remaining)
        
        # In case effective_max_offset < min_offset, use effective_max_offset
        # Otherwise, randomly sample
        offset = effective_max_offset  # or if you want random:
        # offset = random.randint(self.min_offset, effective_max_offset)

        img2_idx = local_idx + offset
        
        img2_path = seq[img2_idx]

        try:
            img1 = Image.open(img1_path).convert("RGB")
            img2 = Image.open(img2_path).convert("RGB")
        except Exception as e:
            print(f"Error loading images: {img1_path} or {img2_path}: {e}")
            # Optionally, raise an error here or return a default dummy tensor
            raise e
        
        return self.transform(img1), self.transform(img2)



In [9]:
import os
from torch.utils.data import DataLoader, random_split
from torchvision import transforms

# Assuming CameraSequentialPairs is defined as in the previous code block.
# For example:
# class CameraSequentialPairs(Dataset):
#     ... (code from previous message) ...

def get_camera_dataloaders(data_root="./dataset_00", batch_size=8, min_offset=5, max_offset=30, num_workers=8):
    # For training, we use the "train" subfolder within the data root.
    train_dataset = CameraSequentialPairs(
        root=os.path.join(data_root, "train"),
        transform=transforms.ToTensor(),
        min_offset=min_offset,
        max_offset=max_offset
    )
    
    # Similarly, for validation, use the "val" subfolder.
    val_dataset = CameraSequentialPairs(
        root=os.path.join(data_root, "val"),
        transform=transforms.ToTensor(),
        min_offset=min_offset,
        max_offset=max_offset
    )
    
    # Optionally, print out dataset sizes for debugging
    print("Training dataset pairs:", len(train_dataset))
    print("Validation dataset pairs:", len(val_dataset))
    
    # Build DataLoaders with shuffling for training and no shuffle for validation.
    train_loader = DataLoader(
        train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers
    )
    
    val_loader = DataLoader(
        val_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers
    )
    
    return train_loader, val_loader

# Example usage:
train_loader, val_loader = get_camera_dataloaders(
    data_root="./dataset_00", batch_size=16, min_offset=1, max_offset=5, num_workers=8
)

# For testing, iterate over one batch:
for img1, img2 in train_loader:
    print("Batch shapes:", img1.shape, img2.shape)
    break


Training dataset pairs: 199998
Validation dataset pairs: 19998
Batch shapes: torch.Size([16, 3, 128, 128]) torch.Size([16, 3, 128, 128])


In [10]:
train_loader, val_loader = get_camera_dataloaders(data_root="./dataset_00", batch_size=16)
print(len(train_loader))
for img1, img2 in train_loader:
    print(img1.shape, img2.shape)  # e.g., [16, 3, H, W]
    break


Training dataset pairs: 199998
Validation dataset pairs: 19998
12500
torch.Size([16, 3, 128, 128]) torch.Size([16, 3, 128, 128])


In [11]:
# trainer = L.Trainer(max_epochs=200, callbacks=[AlphaScheduler()])
# model = LitKeypointDetector(KeypointPredictor(100), ImageEncoder(), ImageDecoder(64, 100))

# trainer.fit(model, train_dataloaders=train_loader, val_dataloaders=val_loader)


In [12]:
encoder_layer = nn.TransformerEncoderLayer(d_model=200, nhead=8, batch_first=True)
transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=6)
# out = transformer_encoder(test)

In [13]:
class SequenceKeypointDetector(nn.Module):
    def __init__(self, keypoint_detector):
        super().__init__()
        self.keypoint_detector = keypoint_detector  # This is your existing 4D keypoint model

    def forward(self, x):
        """
        Args:
          x: input tensor of shape (B, T, C, H, W)
        Returns:
          Output: tensor of shape (B, T, num_keypoints, 2)
        """
        B, T, C, H, W = x.shape
        # Flatten batch and sequence dimensions
        x_flat = x.view(B * T, C, H, W)
        # Process with keypoint detector (assumed to output (B*T, num_keypoints, 2))
        keypoints_flat = self.keypoint_detector(x_flat)[0]  # For example, if keypoint_detector returns (grid_keypoints, soft_keypoints, heatmaps)
        # Reshape the output back to (B, T, num_keypoints, 2)
        keypoints = keypoints_flat.view(B, T, -1, 2)
        return keypoints

In [14]:

from models.keypoints.KeypointPredictor import KeypointPredictor
from models.keypoints.ImageEncoder import ImageEncoder
from models.keypoints.ImageDecoder import ImageDecoder

seq_detector = SequenceKeypointDetector(KeypointPredictor(100))
# Sample input: (64, 100, 3, 128, 128)
input_tensor = torch.randn(8, 100, 3, 128, 128)
output_keypoints = seq_detector(input_tensor)
transformer_input = output_keypoints.view(8,100,-1)
print(transformer_input.size())  # Should print torch.Size([64, 100, 100, 2])

/home/sgarikipati7/.env/ssta_env/lib/python3.12/site-packages/torchvision/models/_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(


torch.Size([8, 100, 200])


In [15]:
# out = transformer_encoder(transformer_input)

In [16]:
out.size()

torch.Size([1, 64, 32, 32])

In [17]:
from losses.masked_mse import masked_mse
import torchvision.utils as vutils
class LitPredictorT2NOD(L.LightningModule):
    def __init__(self, image_encoder, seq_keypoint_encoder, transformer_encoder, image_decoder):
        super().__init__()
        self.seq_keypoint_encoder = seq_keypoint_encoder
        self.transformer_encoder = transformer_encoder
        self.image_encoder = image_encoder
        self.image_decoder = image_decoder
        D = 200  # flatten dim: T × (K × 2)
        T = 100
        self.pos_embed = nn.Parameter(torch.randn(1, T, D))  # [1, T, D]



        # self.seq_keypoint_encoder.eval()
        
    def training_step(self, batch, batch_idx):
        seq_images, t2nod= batch
        # with torch.no_grad():
        seq_keypoints = self.seq_keypoint_encoder(seq_images)

        B, T, num_kps, _ = seq_keypoints.shape
        flat_seq_keypoints = seq_keypoints.view(B, T, -1)
        flat_seq_keypoints = flat_seq_keypoints + self.pos_embed[:, :T]

        flat_pred_kepoints = self.transformer_encoder(flat_seq_keypoints)

        pred_keypoints = flat_pred_kepoints.view(B,T*num_kps,2)
        last_frame = seq_images[:, -1, :, :, :]
        features = self.image_encoder(last_frame)
        out = self.image_decoder(features, pred_keypoints)
        
        total_loss, bce, t2no_mse, t2nd_mse = masked_mse(out, t2nod)
        self.log("train/masked_mse", total_loss)
        self.log("train/bce", bce)
        self.log("train/t2no_mse", t2no_mse)
        self.log("train/t2nd_mse", t2nd_mse)
        return total_loss
    def validation_step(self, batch, batch_idx):
        seq_images, t2nod = batch
        with torch.no_grad():
            seq_keypoints = self.seq_keypoint_encoder(seq_images)

        B, T, num_kps, _ = seq_keypoints.shape
        flat_seq_keypoints = seq_keypoints.view(B, T, -1)
        flat_pred_kepoints = self.transformer_encoder(flat_seq_keypoints)
        pred_keypoints = flat_pred_kepoints.view(B, T * num_kps, 2)

        last_frame = seq_images[:, -1]  # [B, 3, H, W]
        features = self.image_encoder(last_frame)
        out = self.image_decoder(features, pred_keypoints)  # [B, 3, H, W]

        total_loss, bce, t2no_mse, t2nd_mse, mask = masked_mse(out, t2nod, return_mask=True)

        self.log("val/masked_mse", total_loss)
        self.log("val/bce", bce)
        self.log("val/t2no_mse", t2no_mse)
        self.log("val/t2nd_mse", t2nd_mse)

        if batch_idx == 0:  # Log only the first batch per epoch
            grid = self.visualize_predictions_for_tensorboard(
                last_frame[0],      # RGB image
                t2nod[0],           # [2, H, W] GT
                out[0],             # [3, H, W] prediction
                mask,             # [1, H, W]
            )
            self.logger.experiment.add_image("val/sample_grid", grid, self.global_step)

    def visualize_predictions_for_tensorboard(self, rgb_img, gt, pred, mask):
        """
        Returns a grid of shape [3, H, 7*W] for logging as an image.
        """
        def to_3ch(x):
            if x.ndim == 2:
                return x.unsqueeze(0).repeat(3, 1, 1)
            elif x.shape[0] == 1:
                return x.repeat(3, 1, 1)
            return x

        def normalize(x):
            return (x - x.min()) / (x.max() - x.min() + 1e-5)

        rgb_img = normalize(rgb_img)

        t2no_mask     = (gt[0] != 1).float()
        gt_t2no       = gt[0]
        gt_t2nd       = gt[1]
        print("GT t2no unique values:", gt[0].unique())
        print("GT t2no mask sum:", (gt[0] != 1).float().unique())
        pred_mask_asis = normalize(torch.sigmoid(pred[0]))
        pred_mask = (torch.sigmoid(pred[0]) > 0.5).float()

        pred_t2no     = pred[1] * mask[0] + (~mask[0])
        pred_t2nd     = pred[2] * mask[0] + (~mask[0])

        tiles = [
            to_3ch(rgb_img),
            to_3ch(t2no_mask),
            to_3ch(normalize(gt_t2no)),
            to_3ch(normalize(gt_t2nd)),
            to_3ch(pred_mask),
            to_3ch(pred_mask_asis),
            to_3ch(normalize(pred_t2no)),
            to_3ch(normalize(pred_t2nd)),
            to_3ch(normalize(pred[1])),
            to_3ch(normalize(pred[2]))
        ]

        grid = vutils.make_grid(tiles, nrow=7)
        return grid

    
    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=1e-3)
        return optimizer
    
    

In [18]:
import torch

def logsumexp_pooling(heatmaps, group_size=100, beta=10.0):
    """
    Consolidate heatmaps using LogSumExp pooling.

    Args:
        heatmaps: Tensor of shape (B, K, H, W) where K = num_keypoints
        num_groups: Number of final heatmaps desired
        beta: Temperature parameter for LogSumExp

    Returns:
        Tensor of shape (B, num_groups, H, W)
    """
    B, K, H, W = heatmaps.shape

    # Reshape: (B, num_groups, group_size, H, W)
    grouped = heatmaps.view(B, K//group_size, group_size, H, W)

    # LogSumExp pooling over the group dimension
    pooled = (1.0 / beta) * torch.logsumexp(beta * grouped, dim=2)  # shape: (B, num_groups, H, W)

    return pooled


In [19]:
# Suppose we have 10,000 keypoint heatmaps of size 32x32
keypoints = torch.randn(8, 20000)  # Batch of 8
keypoints = keypoints.view(8,10000,-1)
features = torch.randn(8, 32, 32, 32)  # Batch of 8


In [20]:

decoder = ImageDecoder(32, 1000, 3, condense=True)
decoder(features, keypoints).size()

torch.Size([8, 3, 128, 128])

In [21]:
import torch
from torch.utils.data import Dataset, DataLoader

class DummySequenceDataset(Dataset):
    def __init__(self, num_samples=100, sequence_length=100, channels=3, height=128, width=128):
        """
        Args:
            num_samples (int): Number of samples in the dataset.
            sequence_length (int): Number of images per sequence.
            channels (int): Number of color channels (typically 3 for RGB).
            height (int): Height of each image.
            width (int): Width of each image.
        """
        self.num_samples = num_samples
        self.sequence_length = sequence_length
        self.channels = channels
        self.height = height
        self.width = width

    def __len__(self):
        return self.num_samples

    def __getitem__(self, idx):
        # Generate a random image sequence: shape [T, 3, 128, 128]
        seq_images = torch.rand(self.sequence_length, self.channels, self.height, self.width)
        # Generate a dummy target image (t2nod) for the final frame: shape [3, 128, 128]
        # For example, you could imagine these values are normalized between 0 and 1.
        t2nod = torch.rand(2, self.height, self.width)
        return seq_images, t2nod

In [22]:
def get_camera_dataloaders(batch_size=8, min_offset=5, max_offset=30, num_workers=8):
    train_dataset = DummySequenceDataset(
        num_samples=100, sequence_length=100, channels=3, height=128, width=128
    )
    
    val_dataset = DummySequenceDataset(
        num_samples=100, sequence_length=100, channels=3, height=128, width=128
    )
    
    print("Training dataset pairs:", len(train_dataset))
    print("Validation dataset pairs:", len(val_dataset))
    
    train_loader = DataLoader(
        train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers
    )
    
    val_loader = DataLoader(
        val_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers
    )
    
    return train_loader, val_loader

train_loader, val_loader = get_camera_dataloaders(
    batch_size=16, min_offset=1, max_offset=5, num_workers=8
)

for each in train_loader:
    break
print(each[0].size(), each[1].size())

Training dataset pairs: 100
Validation dataset pairs: 100
torch.Size([16, 100, 3, 128, 128]) torch.Size([16, 2, 128, 128])


In [23]:

# encoder_layer = nn.TransformerEncoderLayer(d_model=200, nhead=8, batch_first=True)
# transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=6)
    
# trainer = L.Trainer(max_epochs=200, callbacks=[AlphaScheduler()])
# model = LitPredictorT2NOD(ImageEncoder(), SequenceKeypointDetector(KeypointPredictor(100)), transformer_encoder, ImageDecoder(64, 1000, 3, condense=True))

# trainer.fit(model, train_dataloaders=train_loader, val_dataloaders=val_loader)

In [24]:
import os
from PIL import Image
import torch
from torch.utils.data import Dataset
import torchvision.transforms as T

import re

def natural_sort(l):
    return sorted(l, key=lambda s: [int(text) if text.isdigit() else text.lower()
                                    for text in re.split(r'(\d+)', s)])

class SequenceDataset(Dataset):
    def __init__(self, 
                 root_dir, 
                 mode='train', 
                 sequence_length=5,  
                 input_transform=None, 
                 target_transform=None):
        """
        Args:
            root_dir (str): Path to the dataset folder (e.g., 'dataset_00').
            mode (str): 'train' or 'val'.
            sequence_length (int): Number of consecutive images in the input sequence.
            input_transform (callable, optional): Transformation to be applied to each input image.
            target_transform (callable, optional): Transformation to be applied to each target image.
        """
        self.sequence_length = sequence_length
        self.input_transform = input_transform
        self.target_transform = target_transform
        
        self.samples = []  # Will hold dictionaries with sample info.
        mode_dir = os.path.join(root_dir, mode)
        
        # Get each camera directory (e.g., camera_0, camera_1, etc.)
        camera_dirs = sorted([d for d in os.listdir(mode_dir) if os.path.isdir(os.path.join(mode_dir, d))])
        
        for camera in camera_dirs:
            camera_path = os.path.join(mode_dir, camera)
            images_dir = os.path.join(camera_path, 'images')
            t2no_dir   = os.path.join(camera_path, 't2no')
            t2nd_dir   = os.path.join(camera_path, 't2nd')
            
            # List and sort file names in each folder
            image_files = natural_sort(os.listdir(images_dir))
            t2no_files  = natural_sort(os.listdir(t2no_dir))
            t2nd_files  = natural_sort(os.listdir(t2nd_dir))
            
            # Check that all three folders contain the same number of files.
            if not (len(image_files) == len(t2no_files) == len(t2nd_files)):
                raise ValueError(f"Mismatch in file counts in camera folder: {camera}")
            
            num_frames = len(image_files)
            # Only use valid starting indices where the full sequence exists.
            for start_idx in range(num_frames - sequence_length + 1):
                self.samples.append({
                    "images_dir": images_dir,
                    "t2no_dir": t2no_dir,
                    "t2nd_dir": t2nd_dir,
                    "start_idx": start_idx,
                    "file_list": image_files  # Assuming the same ordering applies for all directories.
                })
    
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        sample_info = self.samples[idx]
        images_dir = sample_info["images_dir"]
        t2no_dir   = sample_info["t2no_dir"]
        t2nd_dir   = sample_info["t2nd_dir"]
        start_idx  = sample_info["start_idx"]
        file_list  = sample_info["file_list"]
        
        # Load the input sequence as RGB images.
        input_sequence = []
        for i in range(start_idx, start_idx + self.sequence_length):
            img_path = os.path.join(images_dir, file_list[i])
            # Open the image in RGB mode to retain three channels.
            img = Image.open(img_path).convert('RGB')
            if self.input_transform:
                img = self.input_transform(img)
            else:
                # Default: convert image to tensor (C x H x W) with values in [0, 1]
                img = T.ToTensor()(img)
            input_sequence.append(img)
        
        # Stack into a tensor of shape [sequence_length, 3, H, W]
        input_sequence = torch.stack(input_sequence, dim=0)
        
        # For the target, choose the frame corresponding to the last image in the sequence.
        target_idx = start_idx + self.sequence_length - 1
        t2no_path = os.path.join(t2no_dir, file_list[target_idx])
        t2nd_path = os.path.join(t2nd_dir, file_list[target_idx])
        
        # Load target images in grayscale.
        t2no = Image.open(t2no_path).convert('L')
        t2nd = Image.open(t2nd_path).convert('L')
        
        # Resize the targets to 128x128 pixels.
        resize_transform = T.Resize((128, 128))
        t2no = resize_transform(t2no)
        t2nd = resize_transform(t2nd)

        t2no = torch.from_numpy(np.array(t2no)).float().unsqueeze(0) / 100.0 
        t2nd = torch.from_numpy(np.array(t2nd)).float().unsqueeze(0) / 100.0
        
        # Concatenate the two target images along the channel dimension.
        target = torch.cat([t2no, t2nd], dim=0)  # Expected shape: [2, 128, 128]
        
        return input_sequence, target

# Example usage:
if __name__ == '__main__':
    # Define transforms if desired.
    input_transform = T.Compose([
        # Example: You can add T.Resize, T.RandomCrop, etc.
        T.ToTensor()
    ])
    
    target_transform = T.Compose([
        T.ToTensor()
    ])
    
    # Create dataset instances for training and validation.
    dataset_root = 'dataset_00'  # Replace with your dataset path.
    train_dataset = SequenceDataset(root_dir=dataset_root, 
                                    mode='train', 
                                    sequence_length=20, 
                                    input_transform=input_transform, 
                                    target_transform=target_transform)
    
    val_dataset = SequenceDataset(root_dir=dataset_root, 
                                  mode='val', 
                                  sequence_length=20, 
                                  input_transform=input_transform, 
                                  target_transform=target_transform)
    
    # Test by loading one sample.
    sample_in, sample_target = train_dataset[0]
    print("Input sequence shape:", sample_in.shape)      # Expected: [sequence_length, 3, H, W]
    print("Target shape:", sample_target.shape)            # Expected: [2, 128, 128]


Input sequence shape: torch.Size([20, 3, 128, 128])
Target shape: torch.Size([2, 128, 128])


In [25]:
# import matplotlib.pyplot as plt
# import torch

# # Assuming you already defined SequenceDataset and created an instance
# # For example:
# # dataset_root = 'dataset_00'
# # train_dataset = SequenceDataset(root_dir=dataset_root, mode='train', sequence_length=5, 
# #                                 input_transform=input_transform, target_transform=target_transform)

# # Get one sample from the training dataset
# input_sequence, target = train_dataset[0]

# # Visualize the input sequence images.
# # input_sequence shape: [sequence_length, 3, H, W]
# for i in range(input_sequence.shape[0]):
#     # Retrieve the i-th image, which is a tensor of shape [3, H, W]
#     img_tensor = input_sequence[i]
#     # Convert the tensor to a numpy array and transpose to [H, W, 3] for matplotlib
#     img_np = img_tensor.numpy().transpose(1, 2, 0)
    
#     plt.figure()
#     plt.imshow(img_np)
#     plt.title(f"Input Sequence Frame {i}")
#     plt.axis("off")
#     plt.show()

# # Visualize the target images.
# # target shape: [2, 128, 128]; each channel is a grayscale image.
# # We'll display them side by side in separate figures.
# target_names = ['t2no', 't2nd']
# for j in range(target.shape[0]):
#     target_img = target[j]  # tensor shape [128, 128]
    
#     plt.figure()
#     plt.imshow(target_img.numpy().squeeze(), cmap='gray')
#     plt.title(f"Target Image: {target_names[j]}")
#     plt.axis("off")
#     plt.show()


In [ ]:

DATASET_ROOT = "dataset_00"
SEQ_LEN = 100
BATCH_SIZE = 8
NUM_WORKERS = 8
MAX_EPOCHS = 200

# ------------------------------
# Transforms
# ------------------------------
input_transform = T.Compose([
    T.Resize((128, 128)),  # Or whatever size your model expects
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406],  # ImageNet stats
                std=[0.229, 0.224, 0.225])
])

target_transform = T.Compose([
    T.ToTensor(),
    lambda x: x / 100.0
])

# ------------------------------
# Dataset & DataLoaders
# ------------------------------
train_dataset = SequenceDataset(
    root_dir=DATASET_ROOT,
    mode="train",
    sequence_length=SEQ_LEN,
    input_transform=input_transform,
    target_transform=target_transform
)

val_dataset = SequenceDataset(
    root_dir=DATASET_ROOT,
    mode="val",
    sequence_length=SEQ_LEN,
    input_transform=input_transform,
    target_transform=target_transform
)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

# ------------------------------
# Model Initialization
# ------------------------------

encoder_layer = nn.TransformerEncoderLayer(d_model=200, nhead=8, batch_first=True)

from models.keypoints.KeypointPredictor import KeypointPredictor
from models.keypoints.ImageEncoder import ImageEncoder
from models.keypoints.ImageDecoder import ImageDecoder

seq_detector = SequenceKeypointDetector(KeypointPredictor(100))

image_encoder = ImageEncoder()
checkpoint = "/home/sgarikipati7/packages/SSTA_2025/lightning_logs/version_0/checkpoints/epoch=57-step=725000.ckpt"
seq_keypoint_encoder = SequenceKeypointDetector(KeypointPredictor(100))
transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=6)
image_decoder = ImageDecoder(64,100,3, condense=True)

model = LitPredictorT2NOD(
    image_encoder=image_encoder,
    seq_keypoint_encoder=seq_keypoint_encoder,
    transformer_encoder=transformer_encoder,
    image_decoder=image_decoder
)


trainer = L.Trainer(
    max_epochs=MAX_EPOCHS
)
torch.cuda.empty_cache()
trainer.fit(model, train_loader, val_loader,ckpt_path="/home/sgarikipati7/packages/SSTA_2025/lightning_logs/version_3/checkpoints/epoch=4-step=124880.ckpt")


You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 4090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
Restoring states from the checkpoint path at /home/sgarikipati7/packages/SSTA_2025/lightning_logs/version_3/checkpoints/epoch=4-step=124880.ckpt
/home/sgarikipati7/.env/ssta_env/lib/python3.12/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:362: The dirpath has changed from '/home/sgarikipati7/packages/SSTA_2025/lightning_logs/version_3/checkpoints' to '/home/sgarikipati

Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]GT t2no unique values: tensor([0.0000, 0.0100, 0.0200, 0.0300, 0.0400, 0.0500, 0.0600, 0.0700, 0.0800,
        0.0900, 0.1000, 0.1100, 0.1200, 0.1300, 0.1400, 0.1500, 0.1600, 0.1700,
        0.1800, 0.1900, 0.2000, 0.2100, 0.2200, 0.2300, 0.2400, 0.2500, 0.2600,
        0.2700, 0.2800, 0.2900, 0.3000, 0.3100, 0.3200, 0.3300, 0.3400, 0.3500,
        0.3600, 0.3700, 0.3800, 0.3900, 0.4000, 0.4100, 0.4200, 0.4300, 0.4400,
        0.4500, 0.4600, 0.4700, 0.4800, 0.4900, 0.5000, 0.5100, 0.5200, 0.5300,
        0.5400, 0.5500, 0.5600, 0.5700, 0.5800, 0.5900, 0.6000, 0.6100, 0.6200,
        0.6300, 0.6400, 0.6500, 0.6600, 0.6700, 0.6800, 0.6900, 0.7000, 0.7100,
        0.7200, 0.7300, 0.7400, 0.7500, 0.7600, 0.7700, 0.7800, 0.7900, 0.8000,
        0.8100, 0.8200, 0.8300, 0.8400, 0.8500, 0.8600, 0.8700, 0.8800, 0.8900,
        0.9000, 0.9100, 0.9200, 0.9300, 0.9400, 0.9500, 0.9600, 0.9700, 0.9800,
        0.9900, 1.0000], devic